Arxiv Research Paper Dataset Generation Projection

TEAM MEMBERS

- Naren Garapati (ng658@drexel.edu)
- Milan Varghese(mv644@drexel.edu)
- Daniel Lai (dl3285@drexel.edu)
- Rutvij Hiteshkumar Upadhyay (ru45@drexel.edu)​

In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
import csv
import os 
import time
import io
import PyPDF2

In [3]:
!pipreqs ~/Desktop/

INFO: Not scanning for jupyter notebooks.
<unknown>:17: SyntaxWarning: invalid escape sequence '\d'
<unknown>:44: SyntaxWarning: invalid escape sequence '\d'
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in /Users/narengarapati/Desktop/requirements.txt


In [35]:
!pip3 install arxivscraper

In [ ]:
import arxivscraper
scraper = arxivscraper.Scraper(category='q-fin', date_from='2024-01-01',date_until='2024-12-03')

In [37]:
output = scraper.scrape()

fetching up to  1000 records...
Got 503. Retrying after 30 seconds.
fetching up to  1000 records...
Got 503. Retrying after 30 seconds.
fetching up to  1000 records...
fetching up to  2000 records...
Got 503. Retrying after 30 seconds.
fetching up to  2000 records...
fetching up to  3000 records...
Got 503. Retrying after 30 seconds.
fetching up to  3000 records...
fetching up to  4000 records...
Got 503. Retrying after 30 seconds.
fetching up to  4000 records...
fetching up to  5000 records...
Got 503. Retrying after 30 seconds.
fetching up to  5000 records...
fetching up to  6000 records...
Got 503. Retrying after 30 seconds.
fetching up to  6000 records...
fetching is completed in 318.9 seconds.
Total number of records 5254


In [ ]:
# note that output is basically metadata
output

[{'title': 'thermodynamic constraints on neural dimensions, firing rates, brain   temperature and size',
  'id': '0905.3690',
  'abstract': 'there have been suggestions that heat caused by cerebral metabolic activity may constrain mammalian brain evolution, architecture, and function. this article investigates physical limits on brain wiring and corresponding changes in brain temperature that are imposed by thermodynamics of heat balance determined mainly by na$^{+}$/k$^{+}$-atpase, cerebral blood flow, and heat conduction. it is found that even moderate firing rates cause significant intracellular na$^{+}$ build-up, and the atp consumption rate associated with pumping out these ions grows nonlinearly with frequency. surprisingly, the power dissipated by the na$^{+}$/k$^{+}$ pump depends biphasically on frequency, which can lead to the biphasic dependence of brain temperature on frequency as well. both the total power of sodium pumps and brain temperature diverge for very small fiber d

In [ ]:
# # code to store meta data in case we need it down the line 
# with open('arxiv_metadata.csv', 'w', newline='') as file:
    
#     fieldnames = output[0].keys()
    
#     # Create CSV writer
#     writer = csv.DictWriter(file, fieldnames=fieldnames)
    
#     # Write headers
#     writer.writeheader()
    
#     # Write data rows
#     writer.writerows(output)

In [ ]:
# code to webscrape research papers using the metadata 

def fetch_paper_data(url):
    time.sleep(3)
    
    headers = {
        'User-Agent': 'Educational Bot (na648@drexel.edu)',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9'
    }
    
    try:
        #SENDING API REQUEST FOR THE URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        title = soup.find('h1', class_='title').text.strip()
        abstract = soup.find('blockquote', class_='abstract').text.strip()
        authors = [author.text.strip() for author in soup.find_all('div', class_='authors')]
        
        try:
            #SEARCHING FOR THE PDF IN THE WEBSITE OF THE LINK
            href = soup.find('a', class_='abs-button download-pdf')['href']
            base = 'https://arxiv.org'
            link = base + href
            response = requests.get(link)
            pdf_file = io.BytesIO(response.content)

            try:
                #reading the contents of the pdf
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text()
            except Exception as e:
                print(f"Error extracting PDF text from {url}: {e}")
                text = ""
        except:
            text = ''
        
        #Returning the extracted contents
        return {
            'title': title,
            'abstract': abstract,
            'authors': authors,
            'body': text,
            'url': url
        }
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def process_papers_incrementally(output, csv_path=None):
    #PREPROCESSING THE DOWNLOADED DATA TO GENERATE FINAL DATASET
    if csv_path is None:
        csv_path = os.path.join(os.path.expanduser('~'), 'Downloads', 'research_papers.csv')
    
    # Track processed URLs to avoid duplicates
    processed_urls = set()
    
    # If file exists, read processed URLs
    if os.path.exists(csv_path):
        try:
            with open(csv_path, 'r', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                processed_urls = {row['url'] for row in reader}
        except Exception as e:
            print(f"Error reading existing CSV: {e}")
    
    results = []
    file_exists = os.path.exists(csv_path)
    
    for paper in output:
        url = paper['url']
        if url in processed_urls:
            print(f"Skipping already processed URL: {url}")
            continue
            
        print(f"Processing: {url}")
        data = fetch_paper_data(url)
        
        if data:
            results.append(data)
            
            try:
                mode = 'a' if file_exists else 'w'
                with open(csv_path, mode, newline='', encoding='utf-8') as csvfile:
                    fieldnames = data.keys()
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    
                    if not file_exists:  # Write header only for new file
                        writer.writeheader()
                        file_exists = True
                    
                    writer.writerow(data)
                print(f"Saved data for: {url}")
                processed_urls.add(url)
            except Exception as e:
                print(f"Error saving data for {url}: {e}")
    
    return results


In [42]:
results = process_papers_incrementally(output)

Processing: https://arxiv.org/abs/0905.3690
Saved data for: https://arxiv.org/abs/0905.3690
Processing: https://arxiv.org/abs/1103.2273
Saved data for: https://arxiv.org/abs/1103.2273
Processing: https://arxiv.org/abs/1201.1641
Saved data for: https://arxiv.org/abs/1201.1641
Processing: https://arxiv.org/abs/1204.5203
Saved data for: https://arxiv.org/abs/1204.5203
Processing: https://arxiv.org/abs/1206.3472
Saved data for: https://arxiv.org/abs/1206.3472
Processing: https://arxiv.org/abs/1210.7091
Saved data for: https://arxiv.org/abs/1210.7091
Processing: https://arxiv.org/abs/1211.7317
Saved data for: https://arxiv.org/abs/1211.7317
Processing: https://arxiv.org/abs/1307.4296
Saved data for: https://arxiv.org/abs/1307.4296
Processing: https://arxiv.org/abs/1403.7660
Saved data for: https://arxiv.org/abs/1403.7660
Processing: https://arxiv.org/abs/1407.4992
Saved data for: https://arxiv.org/abs/1407.4992
Processing: https://arxiv.org/abs/1409.7291
Saved data for: https://arxiv.org/ab

unknown widths : 
[0, IndirectObject(2289, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2292, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2294, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2296, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2298, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2300, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2302, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2304, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2306, 0, 4596143296)]
unknown widths : 
[0, IndirectObject(2308, 0, 4596143296)]


Saved data for: https://arxiv.org/abs/1711.03575
Processing: https://arxiv.org/abs/1712.04633
Saved data for: https://arxiv.org/abs/1712.04633
Processing: https://arxiv.org/abs/1801.04057
Saved data for: https://arxiv.org/abs/1801.04057
Processing: https://arxiv.org/abs/1802.07480
Saved data for: https://arxiv.org/abs/1802.07480
Processing: https://arxiv.org/abs/1803.05136
Saved data for: https://arxiv.org/abs/1803.05136
Processing: https://arxiv.org/abs/1803.05275
Saved data for: https://arxiv.org/abs/1803.05275
Processing: https://arxiv.org/abs/1805.10235
Saved data for: https://arxiv.org/abs/1805.10235
Processing: https://arxiv.org/abs/1807.09932
Saved data for: https://arxiv.org/abs/1807.09932
Processing: https://arxiv.org/abs/1808.03386
Saved data for: https://arxiv.org/abs/1808.03386
Processing: https://arxiv.org/abs/1808.10806
Saved data for: https://arxiv.org/abs/1808.10806
Processing: https://arxiv.org/abs/1809.03930
Saved data for: https://arxiv.org/abs/1809.03930
Processing:

incorrect startxref pointer(1)


Saved data for: https://arxiv.org/abs/2004.04229
Processing: https://arxiv.org/abs/2004.08807
Saved data for: https://arxiv.org/abs/2004.08807
Processing: https://arxiv.org/abs/2004.10177
Saved data for: https://arxiv.org/abs/2004.10177
Processing: https://arxiv.org/abs/2005.00498
Saved data for: https://arxiv.org/abs/2005.00498
Processing: https://arxiv.org/abs/2005.05249
Saved data for: https://arxiv.org/abs/2005.05249
Processing: https://arxiv.org/abs/2005.06201
Saved data for: https://arxiv.org/abs/2005.06201
Processing: https://arxiv.org/abs/2006.04519
Saved data for: https://arxiv.org/abs/2006.04519
Processing: https://arxiv.org/abs/2006.06530
Saved data for: https://arxiv.org/abs/2006.06530
Processing: https://arxiv.org/abs/2006.11330
Saved data for: https://arxiv.org/abs/2006.11330
Processing: https://arxiv.org/abs/2006.11975
Saved data for: https://arxiv.org/abs/2006.11975
Processing: https://arxiv.org/abs/2007.01508
Saved data for: https://arxiv.org/abs/2007.01508
Processing:

...

unknown widths : 
[0, IndirectObject(645, 0, 5128143648)]
unknown widths : 
[0, IndirectObject(652, 0, 5128143648)]
unknown widths : 
[0, IndirectObject(645, 0, 5128143648)]
unknown widths : 
[0, IndirectObject(655, 0, 5128143648)]
unknown widths : 
[0, IndirectObject(657, 0, 5128143648)]
unknown widths : 
[0, IndirectObject(659, 0, 5128143648)]
unknown widths : 
[0, IndirectObject(652, 0, 5128143648)]


Error saving data for https://arxiv.org/abs/2211.13943: 'utf-8' codec can't encode character '\ud835' in position 12925: surrogates not allowed
Processing: https://arxiv.org/abs/2211.14396
Saved data for: https://arxiv.org/abs/2211.14396
Processing: https://arxiv.org/abs/2211.15015
Saved data for: https://arxiv.org/abs/2211.15015
Processing: https://arxiv.org/abs/2211.15963
Saved data for: https://arxiv.org/abs/2211.15963
Processing: https://arxiv.org/abs/2212.00386
Saved data for: https://arxiv.org/abs/2212.00386
Processing: https://arxiv.org/abs/2212.00596
Saved data for: https://arxiv.org/abs/2212.00596
Processing: https://arxiv.org/abs/2212.00829
Saved data for: https://arxiv.org/abs/2212.00829
Processing: https://arxiv.org/abs/2212.01191
Saved data for: https://arxiv.org/abs/2212.01191
Processing: https://arxiv.org/abs/2212.02154
Saved data for: https://arxiv.org/abs/2212.02154
Processing: https://arxiv.org/abs/2212.02714
Saved data for: https://arxiv.org/abs/2212.02714
Processing

FloatObject (b'0.000-10332388') invalid; use 0.0 instead


Saved data for: https://arxiv.org/abs/2212.04535
Processing: https://arxiv.org/abs/2212.04740
Saved data for: https://arxiv.org/abs/2212.04740
Processing: https://arxiv.org/abs/2212.04937
Saved data for: https://arxiv.org/abs/2212.04937
Processing: https://arxiv.org/abs/2212.04953
Saved data for: https://arxiv.org/abs/2212.04953
Processing: https://arxiv.org/abs/2212.05354
Saved data for: https://arxiv.org/abs/2212.05354
Processing: https://arxiv.org/abs/2212.06361
Saved data for: https://arxiv.org/abs/2212.06361
Processing: https://arxiv.org/abs/2212.07225
Saved data for: https://arxiv.org/abs/2212.07225
Processing: https://arxiv.org/abs/2212.08145
Saved data for: https://arxiv.org/abs/2212.08145
Processing: https://arxiv.org/abs/2212.09179
Saved data for: https://arxiv.org/abs/2212.09179
Processing: https://arxiv.org/abs/2212.09832
Saved data for: https://arxiv.org/abs/2212.09832
Processing: https://arxiv.org/abs/2212.10249
Saved data for: https://arxiv.org/abs/2212.10249
Processing:

KeyboardInterrupt: 